#### Calling RunPod Endpoint

In [1]:
# Using the same BASE_URL you defined
INTERNAL_PORT = 30000 # Need to expose port from runpod instance editting window 
POD_ID = "ppas13gg82lsdh" # Specific to each runpod instance
BASE_URL = f"https://{POD_ID}-{INTERNAL_PORT}.proxy.runpod.net" # replace "localhost" with this

import requests

prompts = [  
    "What is the capital of France?",
    "Tell me a short joke"
]

response = requests.post(
    f"{BASE_URL}/generate",
    json=prompts 
)

if response.status_code == 200:
    results = response.json()
    print("\nResults:")
    print(results)
else:
    print(f"Error: {response.status_code}")
    print(response.text)
    
    
# Next question is how to convert interleaved text & image data into prompt
# llama3.2-vision-instruct should provide such information right? 



Results:
{'results': ['The capital of France is Paris.', 'A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?"\n\nThe librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."']}
